In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
init_notebook_mode(connected=True)
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pokemons = pd.read_csv('/kaggle/input/pokemon-images-and-types/pokemon.csv')
numbers = []
for i in range(1,pokemons.shape[0]+1):
    numbers.append(i)
pokemons['pkn'] = numbers
IMG_DIR = '/kaggle/input/pokemon-images-dataset/pokemon/pokemon'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(IMG_DIR) if isfile(join(IMG_DIR, f))]

import shutil
train = './train'
test = './test'
val = './val'

shutil.rmtree(train)
shutil.rmtree(val)
shutil.rmtree(test)

***Find matching data from the numbers and pictures***

In [ ]:
import re
dataframe_img = pd.DataFrame([])
images = []
pokemon_number = []
for img in onlyfiles:
    if not re.search('-', img):
        pkn = img.split('.')
        n = re.sub("[^0-9]", "", pkn[0])
        path = IMG_DIR +'/' +str(img)
        images.append(path)
        pokemon_number.append(n)
dataframe_img['images'] = images
dataframe_img['pkn'] = pokemon_number
dataframe_img['pkn'] = dataframe_img['pkn'].astype(int)
dataframe_img['pkn'] = dataframe_img['pkn'].astype(int)
result = pokemons.merge(dataframe_img, left_on='pkn', right_on='pkn')
result.head()

In [ ]:
select = ['Water', 'Grass']
result = result[result['Type1'].isin(select)]

import os
from shutil import copyfile
os.mkdir('train/')
os.mkdir('test/')
os.mkdir('val/')
for class_ in result['Type1'].unique():
    os.mkdir('train/'+str(class_)+'/')
    os.mkdir('test/'+str(class_)+'/')
    os.mkdir('val/'+str(class_)+'/')
    
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    result, result['Type1'],test_size = 0.3, stratify = result['Type1'])

X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size = 0.3,stratify = y_test)

In [ ]:
from shutil import copyfile, copy2

for image,type_  in zip(X_train['images'], y_train):
    copy2(image, 'train/'+type_)

for image,type_ in zip(X_test['images'], y_test):
    copy2(image, 'test/'+type_)
    
for image,type_ in zip(X_val['images'], y_val):
    copy2(image, 'val/'+type_)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator()

train = datagen.flow_from_directory('train/')
test = datagen.flow_from_directory('test/')
val = datagen.flow_from_directory('val/')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization, Lambda
from keras.preprocessing.image import ImageDataGenerator

def build():
    model = Sequential()
    IMAGE_WIDTH = 256
    IMAGE_HEIGHT = 256
    IMAGE_CHANNELS = 3
    model.add(Lambda(lambda x: x, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
    model.add(Conv2D(32, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (2, 2), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax')) 

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

    model.summary()
    return model
model = build()
history = model.fit_generator(train, epochs=10, validation_data=val)

In [ ]:
predict = model.predict_generator(test)

In [ ]:
fig = go.Figure()
epochs = []
for i in range(len(history.history['acc'])):
    epochs.append(i)
fig.add_trace(go.Scatter(x=epochs,y=history.history['acc'], mode='lines',name='train'))
fig.add_trace(go.Scatter(x=epochs,y=history.history['val_acc'], mode='lines',name='val'))
fig.update_layout({
        'plot_bgcolor': 'rgba(0, 0, 0, 0)',
        'paper_bgcolor': 'rgba(0, 0, 0, 0)',
        'width': 500, 
        'height': 400

})
fig.show()

In [ ]:
from sklearn.metrics import classification_report
predict_frame = pd.DataFrame([])
predict = model.predict_generator(test)
predict_frame['category'] = np.argmax(predict, axis=-1)
labels = dict((v,k) for k,v in val.class_indices.items())
predict_frame['category'] = predict_frame['category'].replace(labels)
print(classification_report(y_test, predict_frame['category']))

In [ ]:
datagentrain = ImageDataGenerator(rotation_range=35, 
                                 zoom_range=[0.5, 1.0], 
                                 brightness_range=[0.2,1.5])

datagen = ImageDataGenerator()

train = datagentrain.flow_from_directory('train/')
test = datagen.flow_from_directory('test/')
val = datagen.flow_from_directory('val/')

In [ ]:
model = build()
history = model.fit_generator(train, epochs=50, validation_data=val)


In [ ]:
from sklearn.metrics import classification_report
predict_frame = pd.DataFrame([])
predict = model.predict_generator(test)
predict_frame['category'] = np.argmax(predict, axis=-1)
labels = dict((v,k) for k,v in val.class_indices.items())
predict_frame['category'] = predict_frame['category'].replace(labels)
print(classification_report(y_test, predict_frame['category']))

In [ ]:
from PIL import Image

def show_wrong_classification(y_test, predict, result):
    tmp = result[result.index.isin(y_test.index)]
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))
    i = 0
    j = 0
    for imag, true, pred in zip(tmp['images'], tmp['Type1'], predict):
        if true != pred:
            if i == (nrows-1) and j == ncols:
                break
            if j > (ncols-1):
                i = 1
                j = 0
            img = Image.open(imag)
            fig = plt.figure()
            ax[i,j].imshow(img)
            ax[i,j].set_title(str(pred))
            j+=1

def show_correct_classification(y_test, predict, result):
    tmp = result[result.index.isin(y_test.index)]
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))
    i = 0
    j = 0
    for imag, true, pred in zip(tmp['images'], tmp['Type1'], predict):
        if true == pred:
            if i == (nrows-1) and j == ncols:
                break
            if j > (ncols-1):
                i = 1
                j = 0
            img = Image.open(imag)
            fig = plt.figure()
            ax[i,j].imshow(img)
            ax[i,j].set_title(str(pred))
            j+=1

In [ ]:
show_wrong_classification(y_test, predict_frame['category'], result)

show_correct_classification(y_test, predict_frame['category'], result)

In [ ]:
def show_all_classification(y_test, predict, result):
    tmp = result[result.index.isin(y_test.index)]
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))
    i=0
    j=0
    for imag, true, pred in zip(tmp['images'], tmp['Type1'], predict):
        if i == (nrows-1) and j == ncols:
            break
        if j > (ncols-1):
            i = 1
            j = 0
        img = Image.open(imag)
        fig = plt.figure()
        ax[i,j].imshow(img)
        ax[i,j].set_title(str(pred)+str(i)+str(j))
        j+=1
            
show_all_classification(y_test, predict_frame['category'], result)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,3,figsize=(15,10))
k =0
import cv2
list_b =[]
list_r = []
list_g = []
from tqdm import tqdm
for type_ in tqdm(result['Type1'].unique()):
    tmp = result[result['Type1']==type_]
    for img in tmp['images']:
        img = cv2.imread(img)
        b, g, r = cv2.split(img)
        color = 'blue'
        for i in b:
            for j in i:
                if j != 0:
                    list_b.append(j)
        color = 'green'
        for i in g:
            for j in i:
                if j != 0:
                    list_g.append(j)
        color = 'red'
        for i in r:
            for j in i:
                if j != 0:
                    list_r.append(j)
    sns.distplot(list_g, ax=axes[k, 0], color='g')
    sns.distplot(list_b, ax=axes[k, 1], color='b')
    sns.distplot(list_r, ax=axes[k, 2], color='r')
    axes[k, 0].set_title('Pokemon type color channel ' + type_)
    if type_ =='Grass':
        list_g_f = list_g
        list_b_f = list_b
        list_r_f = list_r
    else:
        list_g_w = list_g
        list_b_w = list_b
        list_r_w = list_r
    list_b =[]
    list_r = []
    list_g = []
    
    k += 1

In [ ]:
def show_all_classification(y_test, predict, result):
    tmp = result[result.index.isin(y_test.index)]
    nrows = 3
    ncols = 3
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 20))
    i=0
    j=0
    for imag, true, pred in zip(tmp['images'], tmp['Type1'], predict):
        if i == (nrows-1) and j == ncols:
            break
        if j > (ncols-1):
            i = 1
            j = 0
        img = Image.open(imag)
        fig = plt.figure()
        ax[i,j].imshow(img)
        ax[i,j].set_title(str(pred)+str(i)+str(j))
        j+=1
            
show_all_classification(y_test, predict_frame['category'], result)

In [ ]:
model.save("./my_h5_model.h5")